In [1]:
from sklearn.preprocessing import Normalizer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet
import lightgbm as lg
from sklearn.svm import LinearSVC
import pandas as pd 
import numpy as np
import pickle
import os
from sklearn.model_selection import GridSearchCV as  gv
import optuna

C:\Users\Arda\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### INSERTING DATA

In [2]:
data_dir="C:/Users/Arda/Downloads/Upwork/Portfolio/Betting_Asistant/data/"
data=pd.read_csv("{}datamart.csv".format(data_dir))

C:\Users\Arda\AppData\Local\Temp\ipykernel_16224\3281985020.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("{}datamart.csv".format(data_dir))


In [3]:
data= data[data["fixture.status.short"]=="FT"].reset_index(drop=True)

In [4]:
data[data["league.season"]==2024].groupby(["league.name","league.country"])["fixture.id"].count()

league.name          league.country
2. Bundesliga        Germany           364
A-League             Australia         199
Allsvenskan          Sweden            400
Bundesliga           Austria           132
                     Germany           362
Championship         England           740
Eerste Divisie       Netherlands       456
Ekstraklasa          Poland            310
Eliteserien          Norway            400
Eredivisie           Netherlands       358
HNL                  Croatia           176
Indian Super League  India             247
J1 League            Japan             660
Jupiler Pro League   Belgium           376
K League 1           South-Korea        90
La Liga              Spain             274
League One           England           776
League Two           England           664
Ligue 1              France            314
Ligue 2              France            360
Paulista - A1        Brazil            112
Premier League       England           280
                  

In [5]:
feature_data=data.iloc[:,28:]
feature_data

,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,Ball_Possession_Rate,Yellow_Cards,Red_Cards,Total_Passes,...,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal
0,1,4.0,5.0,9.0,3.0,2.5,0.500,1.0,0.0,493.0,...,0,0,1,0,0,0,1,0,0,1.0
1,0,5.0,3.0,11.5,4.5,2.0,0.420,2.0,0.0,338.0,...,0,0,1,0,0,0,1,0,0,1.0
2,1,2.0,5.0,8.0,3.0,1.5,0.440,1.5,0.0,416.0,...,0,0,1,0,0,0,1,0,0,4.0
3,0,5.0,4.5,11.0,5.0,1.0,0.425,2.0,0.0,340.5,...,0,1,0,0,0,0,0,1,0,0.0
4,1,2.5,6.0,11.0,4.5,1.5,0.345,1.5,0.0,261.5,...,0,0,1,0,0,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1,2.0,3.5,16.5,4.0,1.5,0.395,2.0,0.0,306.5,...,0,0,1,0,0,0,0,0,1,1.0
132757,0,5.5,5.5,11.0,4.5,1.0,0.545,1.0,0.0,417.0,...,0,1,0,0,0,0,0,0,1,2.0
132758,1,4.5,6.5,10.5,6.5,1.0,0.580,2.5,0.0,491.5,...,0,1,0,0,0,1,0,0,0,1.0
132759,1,4.5,6.0,12.5,6.0,4.0,0.595,3.5,0.0,472.5,...,0,0,0,1,0,0,0,1,0,2.0


In [6]:
#We will normalize feature which's range over 1

In [7]:

feature_data_v1=pd.concat([data[["fixture.id","league.id","league.season","team_id"]],
                            feature_data],axis=1)


In [8]:
#Distrubition of goals are skewed to rigth
#In order to ballance distrubition with out damaging cause of the project
# We will assing 6 to over 6 goals

#### Train Test 

In [9]:
fixtures=feature_data_v1["fixture.id"].unique()
fix_size= len(fixtures)
fix_size

66569

In [10]:
train= feature_data_v1[data["fixture_date"]<="2025-01-31"]

test=feature_data_v1[data["fixture_date"]>"2025-01-31"]
test

,fixture.id,league.id,league.season,team_id,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,...,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal
123417,1202742,210,2024,1475,1,5.0,6.5,14.0,4.0,2.0,...,0,0,0,1,0,0,0,0,1,1.0
123418,1202742,210,2024,1018,0,2.5,5.5,14.5,2.0,0.5,...,0,1,0,0,0,0,0,1,0,1.0
123419,1202743,210,2024,1017,1,5.5,2.0,14.0,5.5,2.0,...,0,1,0,0,0,0,0,0,1,3.0
123420,1202743,210,2024,608,0,3.5,7.0,11.5,5.0,1.0,...,0,1,0,0,0,0,0,0,1,2.0
123421,1202744,210,2024,1483,1,3.5,4.5,10.5,3.5,1.0,...,1,0,0,0,1,0,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1327719,207,2024,1013,1,2.0,3.5,16.5,4.0,1.5,...,0,0,1,0,0,0,0,0,1,1.0
132757,1327720,207,2024,565,0,5.5,5.5,11.0,4.5,1.0,...,0,1,0,0,0,0,0,0,1,2.0
132758,1327720,207,2024,551,1,4.5,6.5,10.5,6.5,1.0,...,0,1,0,0,0,1,0,0,0,1.0
132759,1327721,207,2024,606,1,4.5,6.0,12.5,6.0,4.0,...,0,0,0,1,0,0,0,1,0,2.0


In [11]:
lmodel=lg.LGBMRegressor()

In [12]:
input_cols=train.columns[4:-1]
input_cols

Index(['Home', 'Shots_on_Goal', 'Shots_off_Goal', 'Foul', 'Corner_Kicks',
       'Offsides', 'Ball_Possession_Rate', 'Yellow_Cards', 'Red_Cards',
       'Total_Passes',
       ...
       'Rank_Name_last_2_3_Oppose', 'Rank_Name_title_Oppose',
       'Rise_Need_1_point_needed_Oppose', 'Rise_Need_3_point_for_catch_Oppose',
       'Rise_Need_3_point_for_chase_Oppose', 'Rise_Need_top_Oppose',
       'Fall_Prevent_1_point_for_run_Oppose',
       'Fall_Prevent_3_point_for_run_Oppose', 'Fall_Prevent_bottom_Oppose',
       'Fall_Prevent_comfort_Oppose'],
      dtype='object', length=113)

In [13]:
otrain,otest=train_test_split(train,test_size=0.3,random_state=42)

In [14]:
target= "Goal"

In [15]:
models= {"lgbm":lg.LGBMRegressor,
         "elastic":ElasticNet,
         "lr":LinearRegression
         }

In [16]:


def try_model(trymod):
    
    
        
    trymod.fit(y=otrain[target], X=otrain[list(input_cols)])
    pred=trymod.predict(otest[list(input_cols)])
    pred[pred<0]=0
   

    return np.mean(abs(pred-otest[target]).sum())



Models

In [17]:

def model_obj(name):
    def objective(trial):

        if name=="lgbm":
            result=try_model(models["lgbm"]( 
                boosting_type=trial.suggest_categorical("boosting_type",
                                                        ["gbdt","dart"]),

                num_leaves= trial.suggest_int("num_leaves",
                                            3,50),
                max_depth=trial.suggest_int("max_depth",
                                            -1,20),
                learning_rate= trial.suggest_float("learning_rate",
                                                0.001,0.1),
                reg_alpha= trial.suggest_float("reg_alpha",
                                            0.0,0.1),

                reg_lambda= trial.suggest_float("reg_lambda",
                                                0.0,0.1),
                verbosity=-1
            ))

            return result      
        
        elif name=="elastic":
            result=try_model(models["elastic"]( 
                            alpha= trial.suggest_float("alpha",
                                                       0.5,5),
                            l1_ratio=trial.suggest_float("l1_ratio",
                                                         0.1,1),
                            fit_intercept=False

            
                            ) )
        
            return result
        
        
        
    return objective
        
        

In [18]:
list(models.keys())

['lgbm', 'elastic', 'lr']

In [19]:
saves={}
for name in list(models.keys())[:-1]:
    
    study = optuna.create_study(direction="minimize")
    
    study.optimize(model_obj(name)
               , n_trials=30)
    
    saves[name]= study.best_params

[I 2025-04-16 13:40:07,611] A new study created in memory with name: no-name-52969803-28b9-40ff-bed5-1f776b226134
[I 2025-04-16 13:40:08,396] Trial 0 finished with value: 35758.61342307877 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 3, 'max_depth': 13, 'learning_rate': 0.0633660260831781, 'reg_alpha': 0.06622310373458594, 'reg_lambda': 0.049255686573978864}. Best is trial 0 with value: 35758.61342307877.
[I 2025-04-16 13:40:09,017] Trial 1 finished with value: 35560.352603214 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 4, 'max_depth': 9, 'learning_rate': 0.06419366905178964, 'reg_alpha': 0.04903438122976245, 'reg_lambda': 0.04417818235082586}. Best is trial 1 with value: 35560.352603214.
[I 2025-04-16 13:40:09,846] Trial 2 finished with value: 35340.567204745894 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 22, 'max_depth': 6, 'learning_rate': 0.08463779302555802, 'reg_alpha': 0.08302091605877929, 'reg_lambda': 0.014292934243227851}. Best is trial 2 wi

In [20]:
def fit_model(model,mod_dict,train,test):

    
    model_in=model(**mod_dict)
    model_in.fit(y=train[target], X=train[list(input_cols)])
    pred=model_in.predict(test[list(input_cols)])
    pred[pred<0]=0
    pred[pred>6]=6
   
    return np.mean(abs(pred-test[target])),model_in

In [21]:
def pred_model(model,main_data):
    pred=model.predict(main_data[list(input_cols)])
    pred[pred<0]=0
   
    return pred

In [22]:

lg_score,lg_model=fit_model(models["lgbm"],saves["lgbm"],train,test)
lg_preds=pred_model(lg_model,test)

lg_score

0.8593444404120031

In [23]:
ela_score,ela_model=fit_model(models["elastic"],saves["elastic"],train,test)
ela_preds=pred_model(ela_model,test)
ela_score

0.9251791832339312

In [24]:
ens_pred=(lg_preds+ela_preds)/2


In [25]:
abs(ens_pred-test["Goal"]).sum()/test.shape[0]

0.8789836875335414

In [26]:
lr_score,lr_model= fit_model(models["lr"],{},train,test)
lr_preds= pred_model(lr_model,test)
lr_score

0.890824063238219

In [27]:
def combinations(iterable, r):

    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [28]:
pred_dict= {"lr":lr_preds, "ela":ela_preds, "lg":lg_preds}

In [29]:
pkeys=list(pred_dict.keys())

In [30]:
searched=list(iter(combinations(pkeys,2)))+[[i] for i in pkeys]+[pkeys]

In [31]:
wining=[]
best_score=10

for i in  searched:
    final_pred=0
    print(i)
    
    for k in i:
        final_pred+= pred_dict[k]
    
    final_pred=final_pred/len(i)

    score=abs(final_pred-test["Goal"]).sum()/final_pred.shape[0]

    if best_score>score:

        wining=i
        best_score=score

('lr', 'ela')
('lr', 'lg')
('ela', 'lg')
['lr']
['ela']
['lg']
['lr', 'ela', 'lg']


In [32]:
def pred_ensembler(models,pre_dict):
    final_pred=0
    for m in  models:
        final_pred+= pre_dict[m]
    
    return final_pred/len(models)

In [33]:
pred_ensembler(wining,pred_dict)

array([1.2548431 , 1.05245279, 1.16403035, ..., 1.44289373, 1.9078071 ,
       0.85976359])

In [34]:
import pickle as pkl

In [35]:
models.keys()

dict_keys(['lgbm', 'elastic', 'lr'])

In [36]:
fitted_models= {"lg":lg_model, "ela":ela_model, "lr":lr_model}

In [40]:
def model_saver(wining,models):

    best_models={}
    for m in  wining:
        best_models[m]=models[m]
    
    curdir=os.getcwd()[:-9]

    with open(f"{curdir}models\\best_models.pkl","wb") as o:
        
            pkl.dump(best_models,o)

In [41]:
model_saver(wining,fitted_models)

In [42]:
test["pred_ela"]=pred_ensembler(wining,pred_dict)
test["ae"]= test[["pred_ela","Goal"]].apply(lambda x: x["pred_ela"]-x["Goal"] ,axis=1)

C:\Users\Arda\AppData\Local\Temp\ipykernel_16224\2973983054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred_ela"]=pred_ensembler(wining,pred_dict)
C:\Users\Arda\AppData\Local\Temp\ipykernel_16224\2973983054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["ae"]= test[["pred_ela","Goal"]].apply(lambda x: x["pred_ela"]-x["Goal"] ,axis=1)


In [43]:

def up_max(x):
    d=x[x>=0]
    if len(d)>0:
        return d.max()
    else:
        return 0

def down_min(x):
    d=x[x<0]
    if len(d)>0:
        return d.min()
    else:
        return 0

    
    

In [44]:
quantiles=test.groupby(["league.id","team_id",],as_index=False)["ae"].aggregate(["median",up_max,down_min])
quantiles

,league.id,team_id,median,up_max,down_min
0,39,33,-0.023058,1.189601,-1.425668
1,39,34,0.606776,1.043148,-2.766245
2,39,35,0.621186,1.466173,-1.699398
3,39,36,-0.787767,1.336535,-0.993564
4,39,39,-0.256032,0.127679,-0.847490
...,...,...,...,...,...
488,475,7848,0.196167,0.848862,-0.821192
489,475,7865,-0.010097,0.981957,-1.864116
490,475,10018,0.830870,1.026917,-0.918127
491,475,10037,-0.140613,0.839262,-1.079179


In [38]:
quantiles.sort_values("down_min")

,league.id,team_id,median,up_max,down_min
18,39,65,-0.872959,1.125870,-5.778583
100,61,91,-1.302554,0.215032,-5.323576
366,197,619,-1.845065,0.142908,-5.084541
28,40,63,-0.650017,0.760953,-5.060683
397,207,565,-0.968736,1.496620,-4.630328
...,...,...,...,...,...
432,235,596,2.099503,2.099503,0.000000
430,235,555,0.949234,0.949234,0.000000
227,98,293,0.928335,0.928335,0.000000
437,235,1080,0.658768,0.658768,0.000000


In [39]:
test

,fixture.id,league.id,league.season,team_id,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,...,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal,pred_ela,ae
123417,1202742,210,2024,1475,1,5.0,6.5,14.0,4.0,2.0,...,0,1,0,0,0,0,1,1.0,1.219659,0.219659
123418,1202742,210,2024,1018,0,2.5,5.5,14.5,2.0,0.5,...,0,0,0,0,0,1,0,1.0,1.091297,0.091297
123419,1202743,210,2024,1017,1,5.5,2.0,14.0,5.5,2.0,...,0,0,0,0,0,0,1,3.0,1.174724,-1.825276
123420,1202743,210,2024,608,0,3.5,7.0,11.5,5.0,1.0,...,0,0,0,0,0,0,1,2.0,1.263232,-0.736768
123421,1202744,210,2024,1483,1,3.5,4.5,10.5,3.5,1.0,...,0,0,1,0,1,0,0,1.0,1.002645,0.002645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132756,1327719,207,2024,1013,1,2.0,3.5,16.5,4.0,1.5,...,1,0,0,0,0,0,1,1.0,1.042700,0.042700
132757,1327720,207,2024,565,0,5.5,5.5,11.0,4.5,1.0,...,0,0,0,0,0,0,1,2.0,1.022428,-0.977572
132758,1327720,207,2024,551,1,4.5,6.5,10.5,6.5,1.0,...,0,0,0,1,0,0,0,1.0,1.401955,0.401955
132759,1327721,207,2024,606,1,4.5,6.0,12.5,6.0,4.0,...,0,1,0,0,0,1,0,2.0,1.945662,-0.054338


Coming Matches

In [ ]:
data_dir="C:/Users/Arda/Downloads/Upwork/Portfolio/Betting_Asistant/data/"
data=pd.read_csv("{}datamart.csv".format(data_dir))
new_data= data[data["fixture.status.short"]!="FT"].reset_index(drop=True)
new_data

C:\Users\Arda\AppData\Local\Temp\ipykernel_15216\1173197005.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("{}datamart.csv".format(data_dir))


,fixture.id,fixture.timezone,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.season,...,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose,Goal
0,1202777,UTC,Split,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,0,1,0,0,1,0,0,0,NaN
1,1202777,UTC,Split,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,0,1,0,0,0,0,1,0,NaN
2,1202778,UTC,Koprivnica,Not Started,NS,NaN,210,HNL,Croatia,2024,...,1,0,0,0,1,0,1,0,0,NaN
3,1202778,UTC,Koprivnica,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,1,0,0,0,0,0,0,1,NaN
4,1202779,UTC,Zagreb,Not Started,NS,NaN,210,HNL,Croatia,2024,...,0,1,0,0,0,0,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,1362792,UTC,Aalborg,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,0,0,1,0,1,0,0,0,NaN
480,1362793,UTC,Haderslev,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,1,0,0,0,0,0,0,1,NaN
481,1362793,UTC,Haderslev,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,0,1,0,0,0,0,0,1,NaN
482,1362794,UTC,Lyngby,Not Started,NS,NaN,119,Superliga,Denmark,2024,...,0,0,1,0,0,0,1,0,0,NaN


: 

In [46]:
new_data[input_cols].dropna(axis=0)

,Home,Shots_on_Goal,Shots_off_Goal,Foul,Corner_Kicks,Offsides,Ball_Possession_Rate,Yellow_Cards,Red_Cards,Total_Passes,...,Rank_Name_last_2_3_Oppose,Rank_Name_title_Oppose,Rise_Need_1_point_needed_Oppose,Rise_Need_3_point_for_catch_Oppose,Rise_Need_3_point_for_chase_Oppose,Rise_Need_top_Oppose,Fall_Prevent_1_point_for_run_Oppose,Fall_Prevent_3_point_for_run_Oppose,Fall_Prevent_bottom_Oppose,Fall_Prevent_comfort_Oppose
0,0,2.5,5.0,15.0,5.5,1.5,0.520,2.0,0.0,335.5,...,0,0,0,1,0,0,1,0,0,0
1,1,3.5,6.0,13.0,4.0,3.5,0.535,1.5,0.0,379.5,...,0,0,0,1,0,0,0,0,1,0
2,1,3.5,4.5,15.0,3.5,3.5,0.490,2.5,0.0,386.5,...,0,1,0,0,0,1,0,1,0,0
3,0,3.5,7.0,17.0,3.5,2.0,0.520,1.5,0.0,387.0,...,0,0,1,0,0,0,0,0,0,1
4,0,1.5,3.0,13.0,2.0,0.5,0.415,2.5,0.5,353.5,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0,5.5,3.0,11.0,4.0,1.0,0.420,2.0,0.0,405.0,...,0,0,0,0,1,0,1,0,0,0
480,1,3.5,5.0,9.5,3.5,2.0,0.560,0.5,0.0,517.5,...,0,0,1,0,0,0,0,0,0,1
481,0,3.5,2.0,9.5,2.5,1.0,0.410,2.0,0.0,459.5,...,1,0,0,1,0,0,0,0,0,1
482,0,5.5,4.0,15.5,5.0,3.0,0.400,1.0,0.0,373.0,...,1,0,0,1,0,0,0,1,0,0


In [47]:

new_preds=pred_model(lg_model,new_data)
#new_preds=new_preds/3

In [48]:
new_data.columns[20:]

Index(['score.fulltime.away', 'fixture.status.extra', 'league.standings',
       'fixture_date', 'fixture_start_time', 'home_point', 'away_point',
       'team_id', 'Home', 'Shots_on_Goal',
       ...
       'Rank_Name_title_Oppose', 'Rise_Need_1_point_needed_Oppose',
       'Rise_Need_3_point_for_catch_Oppose',
       'Rise_Need_3_point_for_chase_Oppose', 'Rise_Need_top_Oppose',
       'Fall_Prevent_1_point_for_run_Oppose',
       'Fall_Prevent_3_point_for_run_Oppose', 'Fall_Prevent_bottom_Oppose',
       'Fall_Prevent_comfort_Oppose', 'Goal'],
      dtype='object', length=122)

In [49]:
check_data=new_data[["fixture.id","league.id",'league.name', 
          'league.country','league.round',
           'teams.home.name', 'teams.away.name',
           'fixture_date',"team_id","Home"]]
check_data["preds"]=new_preds

C:\Users\Arda\AppData\Local\Temp\ipykernel_15216\214754067.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_data["preds"]=new_preds


In [50]:
check_data_v1=check_data.merge(quantiles)
check_data_v1["Lower_q"]=check_data_v1.preds+check_data_v1["down_min"]
check_data_v1["Upper_q"]= check_data_v1.preds+check_data_v1["up_max"]

In [51]:
up_comming=check_data_v1[check_data_v1.fixture_date<="2025-04-31"]
up_comming_v1=up_comming.groupby(list(up_comming.columns[:7]),as_index=False)[["preds","Lower_q","Upper_q"]].sum()


In [52]:
up_comming_v1[up_comming_v1["league.country"]=="Turkey"]

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q
166,1238101,203,Süper Lig,Turkey,Regular Season - 29,Sivasspor,Adana Demirspor,2.205785,0.268992,4.392473
167,1238102,203,Süper Lig,Turkey,Regular Season - 29,Besiktas,Galatasaray,2.182160,-1.754379,3.781845
168,1238103,203,Süper Lig,Turkey,Regular Season - 29,Antalyaspor,Alanyaspor,2.251280,-1.131179,4.342762
169,1238104,203,Süper Lig,Turkey,Regular Season - 29,Kayserispor,Hatayspor,2.108082,-0.475830,4.111309
170,1238105,203,Süper Lig,Turkey,Regular Season - 29,Konyaspor,Gazişehir Gaziantep,2.393585,-0.482236,4.494510
171,1238106,203,Süper Lig,Turkey,Regular Season - 29,Trabzonspor,Goztepe,2.080791,-0.472906,4.878766
172,1238107,203,Süper Lig,Turkey,Regular Season - 29,Samsunspor,Kasimpasa,2.244075,-2.524738,3.429168
173,1238108,203,Süper Lig,Turkey,Regular Season - 29,Eyüpspor,Istanbul Basaksehir,2.229015,-0.550801,4.528356
174,1238109,203,Süper Lig,Turkey,Regular Season - 29,BB Bodrumspor,Fenerbahce,2.038778,0.395304,4.132933


In [53]:
up_comming_v1.sort_values("preds")

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q
197,1324019,98,J1 League,Japan,Regular Season - 7,Kyoto Sanga,Sanfrecce Hiroshima,0.904456,-0.180730,0.904456
27,1208750,140,La Liga,Spain,Regular Season - 29,Getafe,Villarreal,1.006758,1.006758,1.858665
193,1324015,98,J1 League,Japan,Regular Season - 7,FC Tokyo,Kawasaki Frontale,1.194070,1.194070,2.188481
198,1324020,98,J1 League,Japan,Regular Season - 7,Fagiano Okayama,Yokohama F. Marinos,1.218039,1.218039,2.332934
191,1324013,98,J1 League,Japan,Regular Season - 7,Kashima,Vissel Kobe,1.223132,1.223132,1.796704
...,...,...,...,...,...,...,...,...,...,...
77,1213989,61,Ligue 1,France,Regular Season - 27,Monaco,Nice,3.094512,-4.013318,3.471714
24,1208729,140,La Liga,Spain,Regular Season - 27,Barcelona,Osasuna,3.122251,-0.872275,3.122251
52,1213129,235,Premier League,Russia,Regular Season - 22,Zenit,Rubin,3.182696,3.182696,5.310319
74,1213986,61,Ligue 1,France,Regular Season - 27,Saint Etienne,Paris Saint Germain,3.354871,-2.040704,5.203951


In [54]:
up_comming_v1["odd_up"]=(up_comming_v1["Upper_q"]-2.5)/(up_comming_v1["Upper_q"]-up_comming_v1["Lower_q"])
up_comming_v1["odd_down"]=(2.5-up_comming_v1["Lower_q"])/(up_comming_v1["Upper_q"]-up_comming_v1["Lower_q"])

In [55]:
up_comming_v1.sort_values("odd_up",ascending=False).head(10)

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q,odd_up,odd_down
52,1213129,235,Premier League,Russia,Regular Season - 22,Zenit,Rubin,3.182696,3.182696,5.310319,1.320873,-0.320873
31,1208754,140,La Liga,Spain,Regular Season - 29,Real Madrid,Leganes,2.690251,2.690251,5.230722,1.074888,-0.074888
150,1231964,94,Primeira Liga,Portugal,Regular Season - 27,Estrela,Sporting CP,2.480486,2.480486,3.674298,0.983654,0.016346
194,1324016,98,J1 League,Japan,Regular Season - 7,Albirex Niigata,Gamba Osaka,2.336597,2.336597,4.248880,0.914551,0.085449
102,1217250,42,League Two,England,Regular Season - 26,Carlisle,Milton Keynes Dons,2.379650,2.258204,5.081033,0.914343,0.085657
55,1213132,235,Premier League,Russia,Regular Season - 22,CSKA Moscow,Makhachkala,2.366363,2.366363,3.437458,0.875233,0.124767
190,1324012,98,J1 League,Japan,Regular Season - 7,Cerezo Osaka,Urawa,2.398163,2.358543,3.475255,0.873328,0.126672
56,1213133,235,Premier League,Russia,Regular Season - 22,Dynamo,FC Orenburg,2.388390,2.388390,3.261855,0.872221,0.127779
195,1324017,98,J1 League,Japan,Regular Season - 7,Shimizu S-pulse,Shonan Bellmare,2.361620,2.361620,3.443186,0.872056,0.127944
41,1210155,89,Eerste Divisie,Netherlands,Regular Season - 32,Jong PSV,Den Bosch,2.329895,2.125757,3.887287,0.787547,0.212453


In [56]:
up_comming_v1["Lower_q"]=up_comming_v1["Lower_q"].apply(lambda x: 0 if x<0 else x)

In [57]:
up_comming_v1.sort_values("odd_down",ascending=False)

,fixture.id,league.id,league.name,league.country,league.round,teams.home.name,teams.away.name,preds,Lower_q,Upper_q,odd_up,odd_down
148,1231943,94,Primeira Liga,Portugal,Regular Season - 24,GIL Vicente,Benfica,1.579264,1.089509,1.579264,-1.879994,2.879994
197,1324019,98,J1 League,Japan,Regular Season - 7,Kyoto Sanga,Sanfrecce Hiroshima,0.904456,0.000000,0.904456,-1.470296,2.470296
191,1324013,98,J1 League,Japan,Regular Season - 7,Kashima,Vissel Kobe,1.223132,1.223132,1.796704,-1.226169,2.226169
54,1213131,235,Premier League,Russia,Regular Season - 22,Krylia Sovetov,Lokomotiv,2.390169,2.268578,2.396275,-0.812276,1.812276
27,1208750,140,La Liga,Spain,Regular Season - 29,Getafe,Villarreal,1.006758,1.006758,1.858665,-0.752823,1.752823
...,...,...,...,...,...,...,...,...,...,...,...,...
102,1217250,42,League Two,England,Regular Season - 26,Carlisle,Milton Keynes Dons,2.379650,2.258204,5.081033,0.914343,0.085657
194,1324016,98,J1 League,Japan,Regular Season - 7,Albirex Niigata,Gamba Osaka,2.336597,2.336597,4.248880,0.914551,0.085449
150,1231964,94,Primeira Liga,Portugal,Regular Season - 27,Estrela,Sporting CP,2.480486,2.480486,3.674298,0.983654,0.016346
31,1208754,140,La Liga,Spain,Regular Season - 29,Real Madrid,Leganes,2.690251,2.690251,5.230722,1.074888,-0.074888
